In [ ]:
import glob
import json
import os
from pathlib import Path

import numpy as np
import h5py
import tqdm
from skimage.io import imsave
from skimage.exposure import rescale_intensity

In [ ]:
root_dir = '/scratch/hoerl/auto_sir_dna_comp/'

In [ ]:
with open(os.path.join(root_dir, 'sorting20211115.json'), 'r') as fd:
    sorting_dict = json.load(fd)
# sorting_dict

In [ ]:
in_files = glob.glob(os.path.join(root_dir, '*/*/*.h5'))
stem_map = {Path(in_file).stem: in_file for in_file in in_files}

In [ ]:
classes = ['good', 'bad']
out_prefix = 'sorting_20211115_pngs'

for cls in classes:
    
    outdir = Path(root_dir) / out_prefix / cls
    if not outdir.exists():
        outdir.mkdir(parents=True)
    

    for stem, dataset in tqdm.tqdm(sorting_dict[cls]):

        with h5py.File(stem_map[stem], 'r') as fd:
            img = fd[f'/experiment/{dataset}/0/0'][...].squeeze()
            
            img = rescale_intensity(img, tuple(np.quantile(img, (0.02, 0.995))), 'uint8').astype(np.uint8)
    
        imsave(str(outdir / f'{stem}_{dataset}.png'), img)